# Common Functional Design Patterns 

# Carrying : One Argument per Function

`add(1, 2, 3)` =>`add(1)(2)(3)`

#### Let's take a simple function

Most functions, even simple ones, take multiple arguments

In [1]:
def add(a, b, c):
    
    return a + b + c

print(add(10,100,1000))

1110


#### Binding function arguments

But we can 'bind' arguments to a function, so that we end up with a function that takes one argument less than the original function. This is done with `functools.partial()`.

In [2]:
from functools import partial

add_10 = partial(add, 10)
add_10_100 = partial(add_10, 100)
print(add_10_100(1000))

1110


#### Currying

*Currying* is a specific kind of argument binding, in which we create a sequence of functions that each take exactly one argument. In Python, you can implement this elegantly with a decorator.

In [3]:
from inspect import signature

def curry(fnc):
    
    def inner(arg):
        
        # check how many arguments of function take
        if len(signature(fnc).parameters) == 1:
            return fnc(arg)
        return curry(partial(fnc, arg))
    
    return inner
        
    
@curry
def add(a, b, c):
    
    return a + b + c


add_10 = add(10)
add_10_100 = add_10(100)
print(add_10_100(1000))

1110


In [4]:
print(add(10)(100)(1000))

1110


In [5]:
def test(a, b, c, *args, **kwargs):
  pass
print(list(signature(test).parameters))
len(signature(test).parameters)

['a', 'b', 'c', 'args', 'kwargs']


5

# Monads – Variables That Decide How They Should Be Treated

Let's start with some function, `camelcase()`, which transform `'a_string_like_this'` into `'AStringLikeThise'`.

In [6]:
def camelcase(s):
    
    return ''.join([w.capitalize() for w in s.split('_')])

print(camelcase('some_function'))

SomeFunction


#### The Maybe monad

The Maybe monad consists of two kinds of data, which are typically called `Just` and `Nothing`. They both behave very simply:

- When a function is bound to a `Just` value, the function is simply executed, and the result is stored in a new `Just` value.
- When a function is bound to a `Nothing` value, the function is bypassed, and `Nothing` is returned right away.
- In additon, when a function generates an error, it returns a `Nothing` value.

See how similar this is to `nan` behavior?

In [7]:
class Just:
    
    def __init__(self, value):
        
        self._value = value
        
    def bind(self, fnc):
        
        try:
            return Just(fnc(self._value))
        except:
            return Nothing()
    
    def __repr__(self):
        
        return self._value
    


class Nothing:
    
    def bind(self, fnc):
        
        return Nothing()
    
    def __repr__(self):
        
        return 'Nothing'
    
    
print(Just('some_function').bind(camelcase))
print(Nothing().bind(camelcase))
print(Just(10).bind(camelcase))

SomeFunction
Nothing
Nothing


#### The List monad

The `List` monad stores a list of values. When it is bound to a function, each value is passed onto the function separately, and the result is stored as another `List`.

In [8]:
class List:
    
    def __init__(self, values):
        
        self._values = values
        
    def bind(self, fnc):
        
        return List([fnc(value) for value in self._values])
    
    def __repr__(self):
        
        return str(self._values)
    
    
List(['some_text', 'more_text']).bind(camelcase)

['SomeText', 'MoreText']

# Memoization – Remembering Results

#### Some functions are expensive

Let's consider a function that can take a long time to execute.

In [9]:
def prime(n):
    """Return largest prime number that <= n"""
    for i in range(n, 0, -1):
        # (i // x) != (i / x) ===> i is dividable by x
        # if all([(i // x) != (i / x) for x in range(i-1, 1, -1)]):
        if all([i % x != 0 for x in range(i-1, 1, -1)]):
            return i
        
print(prime(1000000))

999983


#### Caching

We can speed up function calls by storing the result in a cache.

In [10]:
cache = {}

def cached_prime(n):
    
    if n in cache:
        return cache[n]    
    for i in range(n, 0, -1):
        if all([i % x != 0 for x in range(i-1, 1, -1)]):
            cache[n] = i
            return i
        
print(cached_prime(1000000))
print(cached_prime(1000000))

999983
999983


#### Memoization

Memoization is a type of caching in which return values are stored for specific arguments. Therefore, the implementation above is an example of memoization. But it can be implemented more elegantly using a decorator!

In [11]:
def memoize(fnc):
    
    cache = {}
    
    def inner(*args):
        
        if args in cache:
            return cache[args]
        cache[args] = fnc(*args)
        return cache[args]
    
    return inner


@memoize
def memoized_prime(n):
    
    for i in range(n, 0, -1):
        if all([i % x != 0 for x in range(i-1, 1, -1)]):
            return i
        

print(memoized_prime(1000000))
print(memoized_prime(1000000))

999983
999983
